### 네이버 쇼핑 데이터 이용하여 Apriori 알고리즘 사용

In [ ]:
# 데이터 로드
df = pd.read_table('/content/drive/MyDrive/Project/Machine Learning/corpus_bab2min/naver_shopping.txt',sep='\t', names =['Rating', 'Review_Text'])

# Okt이용하여 형태소 분석. (norm 사용 안함)
okt = Okt()

dataset = []
for i in df['Review_Text']:
  words = okt.nouns(i)
  if len(words) > 1:
    dataset.append(words)

Total_df = pd.DataFrame(list(apriori(dataset, min_support = 0.0001)))
Total_df['length'] = Total_df['items'].apply(lambda x: len(x))

# 결과 저장
path = '/content/drive/MyDrive/Project/Machine Learning/Keyword/'

Total_df.to_csv(path + 'Keyword_원본.csv', index=False)
Total_df.to_excel(path + 'Keyword_원본.xlsx', index=False)

Total_df_2 = Total_df[(Total_df['length'] == 2) & (Total_df['support'] > 0.001)].sort_values(by = 'support', ascending = False)
Total_df_2.to_csv(path + 'Keyword_원본_2.csv', index=False)
Total_df_2.to_excel(path + 'Keyword_원본_2.xlsx', index=False)

In [ ]:
# 원본 데이터 호출
df = pd.read_excel('/content/drive/MyDrive/Project/Machine Learning/Keyword/Keyword_원본.xlsx')
df = df[df['length'] == 2]
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# 원본 데이터에서 items 1과 2를 따로 컬럼으로 만듦. 이 때 단어가 너무 짧으면 NaN 처리함

for num in range(len(df)):
  item1 = df['items'][num].split(',')[0]
  item1 = re.sub('[^A-Z가-힣]', '', item1)

  item2 = df['items'][num].split(',')[1]
  item2 = re.sub('[^A-Z가-힣]', '', item2)

  if len(item1) < 2:
    df.loc[num, 'item1'] = np.NaN
  else:
    df.loc[num, 'item1'] = item1

  if len(item2) < 2:
    df.loc[num, 'item2'] = np.NaN
  else:
    df.loc[num, 'item2'] = item2

In [ ]:
# na를 날려서 df2로 정의
df2 = df.dropna()
df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
# 데이터 수가 너무 많아서 support를 일정량 이상으로 잘라서 df3로 정의함.

df3 = df2[df2['support']> 0.0006].sort_values(by = 'support', ascending = False)
df3.reset_index(drop=True, inplace=True)
df3

In [ ]:
# 저장
df3.to_excel(path + 'Keyword_전처리_1.xlsx', index=False)

### 네트워크 분석

#### 전체 네트워크 분석 및 태그 자료 분석

In [ ]:
!pip install xlsxwriter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import collections

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 7.2 MB/s eta 0:00:00


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Project/Machine Learning/Naver_Shopping_KoBERT_Senti_2.xlsx')
correlation_matrix = df.corr().round(2)

In [ ]:
# 태깅된 데이터 호출
df_apriori = pd.read_excel('/content/drive/MyDrive/Project/Machine Learning/Keyword_전처리_1.xlsx')
df_apriori_2 = df_apriori[(df_apriori['item1 - tag'] !=0) & (df_apriori['item2 - tag'] !=0)] # 태그가 0인 데이터는 제거
df_apriori_2.reset_index(inplace=True, drop=True)

In [ ]:
# 튜블 만들어서 그래프 형성
item_tuple_list = []
for j, i in enumerate(df_apriori['items']):
  a = i.split("'")
  item_1 = a[1]
  item_2 = a[3]
  item_tuple = (a[1], a[3])
  item_tuple_list.append(item_tuple)

G = nx.Graph()
G.add_edges_from(item_tuple_list)

In [ ]:
# 그래프 edge에 support로 가중치 부여
for j, i in enumerate(df_apriori['items']):
  a = i.split("'")
  item_1 = a[1]
  item_2 = a[3]
  item_tuple = (a[1], a[3])
  item_tuple_list.append(item_tuple)
  support = df_apriori['support'][j]
  G.edges[item_tuple]['Support'] = support

In [ ]:
# 태그를 어트리뷰션으로 부여. 근데 태그 인코딩 안되어 있어서 오류남

for j, i in enumerate(df_apriori['items']):
  a = i.split("'")
  item_1 = a[1]
  item_2 = a[3]
  item_tuple = (a[1], a[3])
  support = df_apriori['support'][j]
  G.edges[item_tuple]['Support'] = support
  G.nodes[item_1]['Tag'] = df_apriori['item1 - tag'][j]
  G.nodes[item_2]['Tag'] = df_apriori['item2 - tag'][j]

In [ ]:
# 태그로 튜플 만들어서 관계 파악
tag_tuple = []
for i in range(len(df_apriori_2)):
  tag1 = df_apriori_2['item1 - tag'][i]
  tag2 = df_apriori_2['item2 - tag'][i]
  item_tuple = (tag1, tag2)
  tag_tuple.append(item_tuple)

collections.Counter(tag_tuple).most_common()

In [ ]:
# 그래프 형성
nx.write_gexf(G, "/content/drive/MyDrive/Project/Machine Learning/Network/test_1.gexf")

#### 점수 상하위권 나워서 네트워크 분석

In [ ]:
!pip install konlpy
!pip install apyori

from konlpy.tag import Okt
from apyori import apriori
import pandas as pd
import networkx as nx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 데이터 가공
df = pd.read_excel('/content/drive/MyDrive/Project/Machine Learning/Naver_Shopping_KoBERT_Senti_2.xlsx')
df['KoBERT'] = df['KoBERT_Sentiment_Score_1'] + 1
df['Mix'] = (df['Rating'] + df['KoBERT']) / 2
df.to_excel('/content/drive/MyDrive/Project/Machine Learning/Total_Excel.xlsx', index=False)

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Project/Machine Learning/Total_Excel.xlsx')
df_high = df[df['Mix'] > 2.5]
df_low = df[df['Mix'] < 2.50001]

In [ ]:
df_high.reset_index(inplace=True, drop=True)
df_low.reset_index(inplace=True, drop=True)

In [ ]:
# Okt이용하여 형태소 분석. (norm 사용 안함)
okt = Okt()

dataset = []
for i in df_high['Review_Text']:
  words = okt.nouns(i)
  if len(words) > 1:
    dataset.append(words)

Total_df = pd.DataFrame(list(apriori(dataset, min_support = 0.0001)))
Total_df['length'] = Total_df['items'].apply(lambda x: len(x))

# 결과 저장
path = '/content/drive/MyDrive/Project/Machine Learning/Network'

Total_df.to_csv(path + 'Keyword_high.csv', index=False)
Total_df.to_excel(path + 'Keyword_high.xlsx', index=False)

Total_df_2 = Total_df[(Total_df['length'] == 2) & (Total_df['support'] > 0.001)].sort_values(by = 'support', ascending = False)
Total_df_2.to_csv(path + 'Keyword_high_2.csv', index=False)
Total_df_2.to_excel(path + 'Keyword_high_2.xlsx', index=False)

# 결과 저장
path = '/content/drive/MyDrive/Project/Machine Learning/Network'

Total_df.to_csv(path + 'Keyword_high.csv', index=False)
Total_df.to_excel(path + 'Keyword_high.xlsx', index=False)

Total_df_2 = Total_df[(Total_df['length'] == 2) & (Total_df['support'] > 0.001)].sort_values(by = 'support', ascending = False)
Total_df_2.to_csv(path + 'Keyword_high_2.csv', index=False)
Total_df_2.to_excel(path + 'Keyword_high_2.xlsx', index=False)

In [ ]:
# Okt이용하여 형태소 분석. (norm 사용 안함)
okt = Okt()

dataset = []
for i in df_low['Review_Text']:
  words = okt.nouns(i)
  if len(words) > 1:
    dataset.append(words)

Total_df = pd.DataFrame(list(apriori(dataset, min_support = 0.0001)))
Total_df['length'] = Total_df['items'].apply(lambda x: len(x))

# 결과 저장
path = '/content/drive/MyDrive/Project/Machine Learning/Network/' 

Total_df.to_csv(path + 'Keyword_low.csv', index=False)
Total_df.to_excel(path + 'Keyword_low.xlsx', index=False)

Total_df_2 = Total_df[(Total_df['length'] == 2) & (Total_df['support'] > 0.001)].sort_values(by = 'support', ascending = False)
Total_df_2.to_csv(path + 'Keyword_low_2.csv', index=False)
Total_df_2.to_excel(path + 'Keyword_low_2.xlsx', index=False)

In [ ]:
Total_df = pd.read_excel('/content/drive/MyDrive/Project/Machine Learning/Network/Keyword_low.xlsx')

In [ ]:
Total_df_low = Total_df[(Total_df['length'] == 2) & (Total_df['support'] > 0.0002)].sort_values(by = 'support', ascending = False)
Total_df_low.reset_index(inplace=True, drop=True)
Total_df_low

,items,support,ordered_statistics,length
0,"frozenset({'그냥', '반품'})",0.020497,"[OrderedStatistic(items_base=frozenset(), item...",2
1,"frozenset({'제품', '배송'})",0.012846,"[OrderedStatistic(items_base=frozenset(), item...",2
2,"frozenset({'그냥', '배송'})",0.012278,"[OrderedStatistic(items_base=frozenset(), item...",2
3,"frozenset({'교환', '그냥'})",0.011127,"[OrderedStatistic(items_base=frozenset(), item...",2
4,"frozenset({'그냥', '사용'})",0.010287,"[OrderedStatistic(items_base=frozenset(), item...",2
...,...,...,...,...
15413,"frozenset({'달', '이제'})",0.000207,"[OrderedStatistic(items_base=frozenset(), item...",2
15414,"frozenset({'재질', '저'})",0.000207,"[OrderedStatistic(items_base=frozenset(), item...",2
15415,"frozenset({'재고', '처리'})",0.000207,"[OrderedStatistic(items_base=frozenset(), item...",2
15416,"frozenset({'마스크', '반품'})",0.000207,"[OrderedStatistic(items_base=frozenset(), item...",2


In [ ]:
# 연관분석 결과 가져와서 전처리 후 그래프 형성
df = pd.read_excel('/content/drive/MyDrive/Project/Machine Learning/Network/Keyword_high.xlsx')

In [ ]:
Total_df_high = df[(df['length'] == 2) & (df['support'] > 0.0001)].sort_values(by = 'support', ascending = False)
Total_df_high.reset_index(inplace=True, drop=True)
Total_df_high

,items,support,ordered_statistics,length
0,"frozenset({'가격', '대비'})",0.020955,"[OrderedStatistic(items_base=frozenset(), item...",2
1,"frozenset({'비', '가성'})",0.020443,"[OrderedStatistic(items_base=frozenset(), item...",2
2,"frozenset({'가격', '배송'})",0.015876,"[OrderedStatistic(items_base=frozenset(), item...",2
3,"frozenset({'배송', '제품'})",0.015750,"[OrderedStatistic(items_base=frozenset(), item...",2
4,"frozenset({'배송', '포장'})",0.015395,"[OrderedStatistic(items_base=frozenset(), item...",2
...,...,...,...,...
20991,"frozenset({'우선', '좀'})",0.000105,"[OrderedStatistic(items_base=frozenset(), item...",2
20992,"frozenset({'맘', '서랍'})",0.000105,"[OrderedStatistic(items_base=frozenset(), item...",2
20993,"frozenset({'그레이', '하나'})",0.000105,"[OrderedStatistic(items_base=frozenset(), item...",2
20994,"frozenset({'맘', '성분'})",0.000105,"[OrderedStatistic(items_base=frozenset(), item...",2


In [ ]:
# 튜블 만들어서 그래프 형성
item_tuple_list = []
for j, i in enumerate(Total_df_high['items']):
  a = i.split("'")
  item_1 = a[1]
  item_2 = a[3]
  if (len(item_1)>1)&(len(item_2)>1):
    item_tuple = (a[1], a[3])
    item_tuple_list.append(item_tuple)

G = nx.Graph()
G.add_edges_from(item_tuple_list)

In [ ]:
len(G.nodes())

1226

In [ ]:
# 그래프 edge에 support로 가중치 부여
for j, i in enumerate(Total_df_high['items']):
  a = i.split("'")
  item_1 = a[1]
  item_2 = a[3]
  if (len(item_1)>1)&(len(item_2)>1):
    item_tuple = (a[1], a[3])
    item_tuple_list.append(item_tuple)
    support = Total_df_high['support'][j]
    G.edges[item_tuple]['Support'] = support

In [ ]:
# 그래프 형성
nx.write_gexf(G, "/content/drive/MyDrive/Project/Machine Learning/Network/high_graph.gexf")